In [1]:
from texascovid19 import constants
import requests
import pandas as pd
from datetime import date

Fetch each day's data.  The data is "current as of 8 p.m. the day before reporting."

In [2]:
r = requests.get(constants.DSHS_XHR_URL, verify=False)
r.raise_for_status()

/Users/dharbin/work/texascovid19/venv/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'services5.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [3]:
county_data = r.json()['features']

In [4]:
norm_county_data = []
for county in county_data:
        norm_county_data.append({
            'County of Residence': county['attributes']['County'],
            'Deaths': county['attributes']['Fatalities'],
            'Number of Cases': county['attributes']['Positive'],
            'Recoveries': county['attributes']['Recoveries'],
            'Active': county['attributes']['Active']
        })

In [5]:
df = pd.DataFrame(data=norm_county_data)

In [6]:
df

,County of Residence,Deaths,Number of Cases,Recoveries,Active
0,Harris,204,9465,3563,5359
1,Dallas,176,7455,3398,3682
2,Tarrant,123,4447,1470,2759
3,Travis,77,2459,892,1456
4,Potter,23,2129,326,1780
...,...,...,...,...,...
217,Kimble,0,1,0,1
218,Knox,0,1,1,0
219,Mitchell,0,1,1,0
220,Motley,0,1,1,0


In [7]:
df.set_index("County of Residence", inplace=True)

In [8]:
today = date.today().isoformat()

In [9]:
df.to_csv(f"{constants.DATA_PATH}/{today}.csv", header=True, index=True)

Append today's data to timeseries

In [10]:
df_timeseries = pd.read_csv(f"{constants.TIMESERIES_CSV}").set_index("County of Residence")

In [11]:
df_copy = df.copy()

In [12]:
df_copy.rename(columns={"Number of Cases": today}, inplace=True)

In [13]:
for col in df_copy.columns:
    if col != today:
        df_copy.drop(col, axis=1, inplace=True)

In [14]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [15]:
df_timeseries = pd.concat([df_timeseries, df_copy], axis=1).rename_axis('County of Residence')

In [16]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [17]:
df_timeseries.to_csv(constants.TIMESERIES_CSV, header=True, index=True)

Also for deaths

In [18]:
df_timeseries = pd.read_csv(f"{constants.DEATHS_TIMESERIES_CSV}").set_index("County of Residence")

In [19]:
df_copy = df.copy()

In [20]:
df_copy.rename(columns={"Deaths": today}, inplace=True)

In [21]:
for col in df_copy.columns:
    if col != today:
        df_copy.drop(col, axis=1, inplace=True)

In [22]:
if today in df_timeseries.columns:
    df_timeseries.drop(columns=today, inplace=True)

In [23]:
df_timeseries = pd.concat([df_timeseries, df_copy], axis=1).rename_axis('County of Residence')

In [24]:
df_timeseries = df_timeseries.fillna(0).convert_dtypes(convert_integer=True)

In [25]:
df_timeseries.to_csv(constants.DEATHS_TIMESERIES_CSV, header=True, index=True)